In [69]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
import  matplotlib.pyplot as plt

In [66]:
data = fetch_olivetti_faces()
images = data.images.reshape((-1, 1, 64, 64))
labels = np.arange(len(images))
images_tensor = torch.tensor(images, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)
dataset = torch.utils.data.TensorDataset(images_tensor, labels_tensor)

index_to_split = int(0.2 * len(images))
test_set = torch.utils.data.TensorDataset(images_tensor[index_to_split:], images_tensor[index_to_split:])

train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [65]:
# Display a picture of dataset
def show_picture(picture_index: int):
    image = data.images[picture_index]
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

In [70]:
# Each picture of the dataset is square 64px x 64px
picture_size = 64

class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(picture_size * picture_size, picture_size * picture_size)
        self.fc2 = nn.Linear(picture_size * picture_size, picture_size * picture_size)
        self.fc3 = nn.Linear(picture_size * picture_size, picture_size * picture_size)
        self.fc4 = nn.Linear(picture_size * picture_size, picture_size * picture_size)

    def forward(self, x):
        # Forward pass through the layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [68]:
EPOCH = 2

model = CNN_model()
loss_fonct = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCH): #training
    for batch in train_loader:
        optimizer.zero_grad()
        images, labels = batch
        output = model.forward(images.reshape(images.shape[0], 64 * 64))
        loss = loss_fonct(output, labels)
        loss.backward()
        optimizer.step()

total, correct = 0, 0
for image, label in test_set:
    print(image)
    output = model.forward(image.reshape(1, 64 * 64))
    # if (output.argmax(dim=1).item() == label):
    #     correct += 1
    total += 1

print("Accuracy: ", str(correct / total * 100) + "%")

tensor([[[0.5000, 0.4876, 0.5372,  ..., 0.4298, 0.4174, 0.2645],
         [0.4669, 0.4793, 0.5372,  ..., 0.4504, 0.4091, 0.2769],
         [0.4380, 0.5165, 0.5455,  ..., 0.4298, 0.4050, 0.3182],
         ...,
         [0.5950, 0.6033, 0.6157,  ..., 0.2645, 0.2810, 0.2810],
         [0.5331, 0.5826, 0.6033,  ..., 0.2686, 0.2769, 0.2851],
         [0.4835, 0.5207, 0.5661,  ..., 0.2934, 0.2934, 0.2810]]])
tensor([[[0.5289, 0.5413, 0.5785,  ..., 0.1405, 0.0992, 0.0579],
         [0.5083, 0.5455, 0.5620,  ..., 0.1653, 0.0702, 0.0992],
         [0.4793, 0.5579, 0.5826,  ..., 0.1570, 0.0826, 0.0909],
         ...,
         [0.6364, 0.6405, 0.6364,  ..., 0.2603, 0.2273, 0.2645],
         [0.6157, 0.6240, 0.6322,  ..., 0.2479, 0.2355, 0.2769],
         [0.6157, 0.6281, 0.6405,  ..., 0.2645, 0.2479, 0.2645]]])
tensor([[[0.5207, 0.5537, 0.5744,  ..., 0.3264, 0.1446, 0.0826],
         [0.5083, 0.5413, 0.5826,  ..., 0.3595, 0.1240, 0.0868],
         [0.5248, 0.5537, 0.5826,  ..., 0.3760, 0.1529, 0.